In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import timeit
import scipy.io as sio

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from MPC_tools import *

%matplotlib inline

In [5]:
base_dir = '/media/cocoan-gpu1/wani8T/data/SEMIC'
project_type = 'semic'
type1 = 'model02_FIR_SPM_SINGLE_TRIAL'
type2 = 'model02_Overall_FIR_SPM_SINGLE_TRIAL'

In [3]:
dir_path_list, filenames_list = making_file_path(base_dir, project_type, type1, type2)

In [4]:
print(dir_path_list.shape)
print(dir_path_list)

(0,)
[]


In [ ]:
print(len(filenames_list[59]))

In [ ]:
full_path_list = making_full_path_list(dir_path_list, filenames_list)

In [ ]:
print(len(full_path_list[10]))

In [ ]:
total_index = np.arange(0,118)
total_index

In [ ]:
load_index = np.delete(total_index, [17,55,56,57,58,75,114,115,116,117])
load_index

In [ ]:
data_num = [0]

In [ ]:
X, Y = load_nii(project_type, load_index, full_path_list, data_num)

In [ ]:
X = flatten_nii(X)

In [ ]:
y = pd.DataFrame(data=Y, dtype=np.float32)

# Logistic Regression Model

In [2]:
X = np.load('/home/cocoan-gpu1/Desktop/SEMIC_sub1to55_num1to50_stim_control_flatten.npy')
print(X.shape)
y = pd.read_csv('/home/cocoan-gpu1/Desktop/SEMIC_sub1to55_num1to50_stim_control_pandas.csv')
print(y.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/home/cocoan-gpu1/Desktop/SEMIC_sub1to55_num1to50_stim_control_flatten.npy'

In [ ]:
y = y['0']
y.shape

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [5]:
del X

In [6]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test, predictions))

/home/cocoan-gpu1/anaconda3/envs/tf3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98       563
         1.0       0.97      0.98      0.98       537

   micro avg       0.98      0.98      0.98      1100
   macro avg       0.98      0.98      0.98      1100
weighted avg       0.98      0.98      0.98      1100



In [7]:
conf_mat = confusion_matrix(y_test, predictions)
print(conf_mat)
acc = (conf_mat[0,0]+conf_mat[1,1])/conf_mat.sum()
print(acc)

[[546  17]
 [  9 528]]
0.9763636363636363


# Load new data

In [ ]:
load_new_index=[55,56,57,58,114,115,116,117]
new_X, new_Y = load_nii(load_new_index, full_path_list, 50)
new_X = flatten_nii(new_X)

In [ ]:
new_Y = pd.DataFrame(data=new_Y, dtype=np.float32)

In [ ]:
predictions_new = logmodel.predict(new_X)
print(classification_report(y_new, predictions_new))

In [ ]:
conf_mat_new = confusion_matrix(y_new, predictions_new)
print(conf_mat_new)

In [ ]:
acc_new = (conf_mat_new[0,0]+conf_mat_new[1,1])/conf_mat_new.sum()
print(acc_new)

In [11]:
logit_coef = logmodel.coef_.flatten()

In [15]:
logit_coef = logit_coef.reshape(79,95,79)

In [18]:
sio.savemat('SEMIC_sub1to55_num50_logit_coef.mat', {'coef':logit_coef})

In [ ]:
from scipy.stats import norm

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

In [ ]:
p = logit_pvalue(logmodel, X_train)

In [ ]:
# test p-values
x = np.arange(10)[:, np.newaxis]
y = np.array([0,0,0,1,0,0,1,1,1,1])
model = LogisticRegression(C=1e30).fit(x, y)
print(logit_pvalue(model, x))

In [ ]:
# compare with statsmodels
import statsmodels.api as sm

In [ ]:
sm_model = sm.Logit(y_train, sm.add_constant(X_train)).fit(disp=0)
print(sm_model.pvalues)

In [ ]:
sm_model.summary()